In [1]:
import numpy as np 
import tensorflow as tf
import pdb
import math 

In [2]:
def real2bits(value,bits=8,radius=1.5):
    binary = np.zeros([bits,])
    step = radius/(2**bits/2)
    if value < 0:
        binary[0] = 1
    integer = int(np.floor(np.abs(value/step)))
    done = False 
    i = bits-1
    while not done: 
        binary[i] = integer % 2
        integer = int(integer/2)
#         pdb.set_trace()
        if integer == 0:
            done = True 
        else:
            i -= 1 
    return binary

In [3]:
def bits2real(bit_array,bits=8,radius=1.5):
    step = radius/(2**(bits-1))
    weights = np.zeros([bits,])
#     weights[0] = -1.0
    for i in range(bits-1):
        weights[bits-i-1] = 2**i
    value = weights.dot(bit_array)*step
    if bool(bit_array[0]):
        value *= -1.0
    return value

In [4]:
def tf_bits2real(X, bits=8,radius=1.5): #FIXME 
    step = radius/(2**(bits-1))
    weight_list = [0]
    for i in range(bits-1):
        power = bits - 2 - i 
        weight_list.append(2**power*step)
    weight_list = np.array(weight_list)
    weight_list = weight_list.reshape([-1,1])
#     pdb.set_trace()
    conversion_tensor = tf.constant(weight_list,
                                   dtype=tf.float32)
#     pdb.set_trace()
    real_output = tf.matmul(X, conversion_tensor,
                            transpose_a=False)
    idx_array = np.zeros([bits,1])
    idx_array[0] = 1
    idx_tensor = tf.constant(idx_array,
                            dtype=tf.float32)
    neg = tf.matmul(X,idx_tensor)
    neg_factor = tf.multiply(neg, tf.constant(-2, dtype=tf.float32))
    scale = neg_factor + 1.0
    real_output = tf.multiply(real_output, scale)
#     tf.cond(tf.cast(neg,tf.bool), )
    return real_output

In [5]:
from tensorflow.python.framework import function

In [6]:
@function.Defun(tf.float32, tf.float32, tf.float32, tf.float32)
def windowgrad(x,center,width,dy):
    lo = center - width/2.0
    hi = center + width/2.0
    lo = 0.0
    hi = 2.0
    lo_mask = tf.greater_equal(x,lo)
    lo_mask = tf.cast(lo_mask,dtype=tf.float32)
    hi_mask = tf.less_equal(x,hi)
    hi_mask = tf.cast(hi_mask,dtype=tf.float32)
    out = tf.multiply(hi_mask, lo_mask)
    out = tf.cast(out, dtype=tf.float32)
    dx = tf.multiply(out,dy)
    return dx, tf.constant(0,dtype=tf.float32), tf.constant(0,dtype=tf.float32) 

@function.Defun(tf.float32, tf.float32, tf.float32, grad_func=windowgrad)
def window(x,center,width):
    lo = center - width/2.0
    hi = center + width/2.0
    lo_mask = tf.greater_equal(x,lo)
    lo_mask = tf.cast(lo_mask,dtype=tf.float32)
    hi_mask = tf.less_equal(x,hi)
    hi_mask = tf.cast(hi_mask,dtype=tf.float32)
    out = tf.multiply(hi_mask, lo_mask)
    out = tf.cast(out, dtype=tf.float32)
    return out 

In [7]:
def basic_MLP_wcustom(X,y,sizes,is_training,bits=8,dropout=False): 
    layers = {}
    layers[0] = tf.layers.dense(X, sizes[0],
                                activation = tf.nn.relu,
                                use_bias=False,
                                )
    if dropout:
        dropout_key = str(0)+'d' 
        layers[dropout_key] = tf.layers.dropout(layers[0],
                                                rate=0.25,
                                                training=is_training)
    for j,  size in enumerate(sizes[1:]): 
        i = j+1
        if dropout: 
            dropout_key_prior = str(i-1)+'d' 
            layers[i] = tf.layers.dense(layers[dropout_key_prior], size,
                                    activation = tf.nn.relu,
                                    use_bias=False,
                                    )
            dropout_key = str(i)+'d' 
            layers[dropout_key] = tf.layers.dropout(layers[dropout_key],
                                                rate=0.25,
                                                training=is_training)
        else:
            layers[i] = tf.layers.dense(layers[i-1], size,
                                    activation = tf.nn.relu,
                                    use_bias=False,
                                    )

    output_value = tf.layers.dense(layers[len(sizes)-1], bits,
                                   activation = None,
                                   use_bias=False,
                                    ) #used relu to guarentee no negative values
#     pdb.set_trace()
    center = tf.ones([1],dtype=tf.float32)
    width = tf.constant(1.0,dtype=tf.float32)
#     output = window(output_value,center,width)
    output = output_value
    return output


In [43]:
def bit_loss(X,y,bits=8,radius=1.5):
    step = radius/(2**(bits-1))
    weight_list = [radius]
    for i in range(bits-1):
        power = bits - 2 - i 
#         weight_list.append(2**power*step)
        weight_list.append(power*step)
    weight_list = np.array(weight_list)
    weight_list = weight_list.reshape([1,-1])
    conversion_tensor = tf.constant(weight_list,
                                   dtype=tf.float32)
#     pdb.set_trace()
    diff = tf.subtract(X, y)
    weighted_diff = tf.multiply(diff, conversion_tensor)
    loss = tf.nn.l2_loss(weighted_diff)
    return loss 

In [45]:
SIZES = [128, 256, 512]
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=[None,1],name='Input')
y = tf.placeholder(tf.float32, shape=[None,4],name='Correct_output')
is_training = tf.placeholder(tf.bool,name='training')
output = basic_MLP_wcustom(X,y,SIZES,is_training,bits=4,dropout=False)
# real_output = tf_bits2real(output,bits=16)
loss = bit_loss(output,y,bits=4,radius=1.5)
mean_loss = tf.reduce_mean(loss)
optimizer = tf.train.AdamOptimizer(1e-3) 
train_step = optimizer.minimize(loss)

In [46]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

xd = np.arange(-1.49, 1.49, 0.01)
# xd = np.ones([1000,1])
# np.random.shuffle(xd)
yd = np.zeros([xd.shape[0],4])
for i, x in enumerate(xd):
    yd[i,:] = real2bits(x, bits=4, radius=1.5)
#     yd[i] = bits2real(yd_, bits=16, radius=1.5)
#     pdb.set_trace()
xd = xd.reshape([-1,1])

In [47]:
for i in range(50000):
    idx = np.random.randint(0,xd.shape[0]-1,1)
    ml, _ = sess.run([mean_loss, train_step],feed_dict={X: xd[idx,:], y: yd[idx]})
    if i % 1000 == 0:
        print(ml)

# total_loss, total_correct = run_model(sess, real_output, loss, xd, yd,
#                                       epochs=1, batch_size=64, print_every=100,
#                                       training=train_step, plot_losses=True)

# iter_cnt,loss,np.sum(corr)/actual_batch_size

1.18761
0.00546597
0.0260251
0.600796
0.0125024
0.00127575
0.00362035
0.00348031
0.0105769
0.000734283
0.412264
0.017302
0.0189731
0.0183478
0.0242712
0.0689911
0.0243588
0.0915553
0.000142189
0.0650866
0.0853367
0.0151026
0.125527
0.0174418
0.0162789
0.00394045
0.00194161
0.740806
0.0103936
0.0119455
0.170052
0.350387
0.0189355
0.122888
0.328072
0.00365347
0.0213579
0.0019197
0.0136743
0.019757
0.00314808
0.27478
0.000249423
0.0103404
0.920155
0.000477844
0.0152864
0.0102045
0.00264682
0.0228537


In [48]:
correct = tf.equal(tf.round(output), y)
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
print(sess.run([accuracy],feed_dict={X: xd, y: yd}))

[0.71476507]


In [49]:
print(sess.run([tf.round(output)],feed_dict={X: xd, y: yd})[0][100:150,:])

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [50]:
print(yd[100:150])

[[ 1.  0.  1.  0.]
 [ 1.  0.  1.  0.]
 [ 1.  0.  1.  0.]
 [ 1.  0.  1.  0.]
 [ 1.  0.  1.  0.]
 [ 1.  0.  1.  0.]
 [ 1.  0.  1.  0.]
 [ 1.  0.  1.  0.]
 [ 1.  0.  1.  0.]
 [ 1.  0.  1.  0.]
 [ 1.  0.  1.  0.]
 [ 1.  0.  1.  0.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
